In [77]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typec

In [78]:
# phase result parser
def flattenArray(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flattenArray(item))
        else:
            result.append(item)
    return result

# Get all possible phases for that components
def getPossiblePhases(db, components):
    possible_phases = []
    for phase_full in db.phases.items():
        # item[0] - name, item[1] - content
        # item[1]: name, constituents, sublattices, model_hints
        found_sublattices = 0
        phase = phase_full[1]

        for sublattice_c in phase.constituents:
            found = False
            for comp in components:
                if len([species.name for species in sublattice_c if species.name == comp]) > 0:
                    found = True
                    break
            if found:
                found_sublattices += 1
            else:
                break

        if found_sublattices == len(phase.constituents):
            possible_phases.append(phase.name)

    possible_phases.sort()
    return possible_phases

# TODO: fix ordered_phases
def filterOrderedPhases(dbf, candidate_phases=None):
    if candidate_phases == None:
        candidate_phases = dbf.phases.keys()
    else:
        candidate_phases = set(
            candidate_phases).intersection(dbf.phases.keys())
    # all_phases = [{"name": phase, "ordered": dbf.phases[phase].model_hints.get('ordered_phase'),'dis':dbf.phases[phase].model_hints.get('disordered_phase')} for phase in candidate_phases if dbf.phases[phase].model_hints.get('ordered_phase') != None]
    # disordered_phases = [dbf.phases[phase].model_hints.get('disordered_phase') for phase in candidate_phases]
    ordered_phases = [dbf.phases[phase].model_hints.get(
        'ordered_phase') for phase in candidate_phases]

    phases = [phase for phase in candidate_phases if
              (phase not in ordered_phases)]
    #  or (phase in disordered_phases and
    # dbf.phases[phase].model_hints.get('ordered_phase') not in candidate_phases)
    return sorted(phases)


def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]


def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def equilibriumRow(conditions, components, temp, amounts):
    calc_components = components.copy()
    calc_components.append('VA')
    row_values = {}
    founded_phases = []
    possible_phases = getPossiblePhases(db, components)
    possible_phases = filterOrderedPhases(db, possible_phases)
    # print(set(possible_phases) - set(filtred_phase))

    tic = time.perf_counter()
    try:
        eq = equilibrium(db, calc_components, possible_phases,
                         conditions, verbose=False)
        founded_phases = flattenArray(np.array(eq.Phase))
    except ValueError as error:
        row_values['Error'] = error
    toc = time.perf_counter()

    row_values = {'Error': '', 'T': temp, 'amounts': amounts, 'P': P, 'Components': compToMask(components), 'phases': phasesToMask(founded_phases),
                  'ellapsed_time': toc - tic, 'possible_phases': phasesToMask(possible_phases)} | row_values

    return row_values

def parseConditions(temp, amounts, components, P=P):
    conditions = {v.T: temp, v.P: P}

    for i in range(len(components)):
        if i != 0:
            conditions[v.X(components[i])] = amounts[i]

    return conditions

def getAmounts(components):
    amounts = [0.05]*len(components)
    for i in range(len(amounts)):
        amounts[i] += random.uniform(0, 1 - sum(amounts))
    amounts[0] += 1 - sum(amounts)
    return amounts

# TODO: make random temperature more stable diffuse
def getTemp():
    return random.randint(300, 3000)


In [79]:
import csv

random.seed(42)
# skip empty case
components = random.sample(all_components, 8) # ['RH', 'CA', 'AL', 'TC', 'GA', 'EU', 'ER', 'CD']
line = {}

# ['TC', 'C', 'SM', 'ZR', 'OS', 'BI', 'PR', 'MG']
with open('ParsedData/equilibrium_result.csv', "a") as csv_file:
    header = ['Error','T','P','Components','phases','ellapsed_time','possible_phases']
    writer = csv.DictWriter(csv_file, fieldnames = header)
    # writer.writeheader()
    writer = csv.writer(csv_file, delimiter=',')
    rnd_state = random.getstate()

    # TODO: iterate over components
    components = random.sample(all_components, 8)
    print(components)
    
    for i in tqdm(range(300)):
        temp = getTemp()
        amounts = getAmounts(components)
        if i < 50:
            continue
        conditions = parseConditions(temp, amounts, components)
        line = equilibriumRow(conditions, components, temp, amounts) 
        writer.writerow(list(line.values()))
    # TODO: Save random state in a file and continue from state
    print('Done!')

['TC', 'C', 'SM', 'ZR', 'OS', 'BI', 'PR', 'MG']


100%|██████████| 300/300 [44:33<00:00,  8.91s/it]

Done!
